In [1]:
from typing import List, Tuple
import numpy as np
import re
import os
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
from datasets import load_dataset
from gensim.models import KeyedVectors

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')

    X_train, y_train = trainset['text'], trainset['category']
    X_val, y_val = valset['text'], valset['category']
    X_test, y_test = testset['text'], testset['category']

    categories = sorted(list(set(y_train)))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories


train_data, val_data, test_data, classes_list = load_sib200_ru()

nlp = spacy.load("ru_core_news_sm", disable=["ner", "parser"])
stemmer = SnowballStemmer("russian")

def normalize_text(s: str, nlp_pipeline: spacy.Language) -> List[str]:
    s = re.sub(r'https?://\S+|www\.\S+', '<URL>', s)
    s = re.sub(r'\S+@\S+', '<EMAIL>', s)
    s = re.sub(r'\d+', '<NUM>', s)
    doc = nlp_pipeline(s)
    tokens = []
    for token in doc:
        if token.is_punct or token.is_space or token.lower_ in STOP_WORDS:
            continue
        lemma = token.lemma_.lower()
        if len(lemma) <= 2:
            continue
        tokens.append(lemma)
    return tokens

print("🧹 Нормализация текста...")
X_train_norm = [normalize_text(it, nlp) for it in train_data[0]]
X_val_norm = [normalize_text(it, nlp) for it in val_data[0]]
X_test_norm = [normalize_text(it, nlp) for it in test_data[0]]

🧹 Нормализация текста...


In [4]:
print("📈 Обучение TF-IDF...")
tfidf = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
tfidf.fit(X_train_norm)
idf_dict = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

print("💾 Загрузка FastText...")
MODEL_PATH_VEC = "./models/cc.ru.300.vec"
w2v = KeyedVectors.load_word2vec_format(MODEL_PATH_VEC, binary=False)

def sentence_vector_tfidf(tokens):
    vecs, weights = [], []
    for w in tokens:
        if w in w2v and w in idf_dict:
            vecs.append(w2v[w])
            weights.append(idf_dict[w])
    if not vecs:
        return np.zeros(w2v.vector_size)
    vecs = np.array(vecs)
    weights = np.array(weights).reshape(-1, 1)
    return np.sum(vecs * weights, axis=0) / np.sum(weights)

print("📐 Векторизация с TF-IDF...")
X_train_vec = np.array([sentence_vector_tfidf(toks) for toks in X_train_norm])
X_val_vec = np.array([sentence_vector_tfidf(toks) for toks in X_val_norm])
X_test_vec = np.array([sentence_vector_tfidf(toks) for toks in X_test_norm])

print("🔻 PCA до 150 компонент...")

pca = PCA(n_components=150, random_state=42)
X_train_vec = pca.fit_transform(X_train_vec)
X_val_vec = pca.transform(X_val_vec)
X_test_vec = pca.transform(X_test_vec)



📈 Обучение TF-IDF...
💾 Загрузка FastText...


/home/toasty/repos/NLP/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


📐 Векторизация с TF-IDF...
🔻 PCA до 150 компонент...


In [5]:
print("🔍 GridSearch LogisticRegression и LinearSVC...")

models = {
    'LogReg': (LogisticRegression(max_iter=3000, random_state=42),
               {
                   'cls__C': [0.01, 0.1, 1, 10, 100, 1000],
                   'cls__solver': ['lbfgs', 'newton-cg', 'saga'],
                   'cls__class_weight': ['balanced', None]
               }),
    'SVM': (LinearSVC(random_state=42),
            {
                'cls__C': [0.01, 0.1, 1, 10],
                'cls__class_weight': ['balanced', None],
                'cls__max_iter': [2000, 5000]
            })
}

best_models = {}

for name, (model, params) in models.items():
    print(f"\n🚀 Обучаем {name}...")
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('cls', model)
    ])

    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        scoring='f1_macro',
        cv=5,
        n_jobs=-1,
        verbose=2
    )

    grid.fit(X_train_vec, train_data[1])
    best_models[name] = grid.best_estimator_

    print(f"\n✅ Лучшие параметры для {name}: {grid.best_params_}")
    print(f"Лучший F1-macro (train CV): {grid.best_score_:.4f}")

    # Валидация
    print(f"\n📊 Валидация ({name}):")
    y_val_pred = grid.predict(X_val_vec)
    print(classification_report(val_data[1], y_val_pred, target_names=classes_list))


🔍 GridSearch LogisticRegression и LinearSVC...

🚀 Обучаем LogReg...
Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=newton-cg; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=None, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=None, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=None, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=lbfgs; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=balanced, cls__solver=newton-cg; total time=   0.0s
[CV] END cls__C=0.01, cls__class_weight=None, cls__solver=lbfgs; total time=   0.0

In [ ]:
# выбираем лучшую по CV
best_model_name = max(best_models.keys(), key=lambda n: best_models[n].score(X_val_vec, val_data[1]))
final_model = best_models[best_model_name]

print(f"\n🏁 Лучшая модель: {best_model_name}")

print("\n🧪 Тест:")
y_test_pred = final_model.predict(X_test_vec)
print(classification_report(test_data[1], y_test_pred, target_names=classes_list))


🏁 Лучшая модель: SVM

🧪 Тест:
                    precision    recall  f1-score   support

     entertainment       0.50      0.42      0.46        19
         geography       0.62      0.88      0.73        17
            health       0.65      0.77      0.71        22
          politics       0.89      0.80      0.84        30
science/technology       0.75      0.78      0.77        51
            sports       0.84      0.84      0.84        25
            travel       0.79      0.65      0.71        40

          accuracy                           0.74       204
         macro avg       0.72      0.74      0.72       204
      weighted avg       0.75      0.74      0.74       204


🧪 валидация:
                    precision    recall  f1-score   support

     entertainment       0.62      0.56      0.59         9
         geography       0.50      0.62      0.56         8
            health       0.82      0.82      0.82        11
          politics       0.92      0.79      0.85  